In [13]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as sm
from sklearn.metrics import mean_squared_error,r2_score,explained_variance_score
from datetime import date
import matplotlib.pyplot as plt
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="p0o9i8u7",
  database="jockey"
)

#file0 = "/Users/apple/Workspace/intelliJ/jockey/regression_20190127.csv"

#dataset0 = pd.read_csv(file0)

#'horse_winCount','jockey_winCount',,'declaredHorseWeight','addedWeight'
#
#features=['draw','horse_winPer','jockey_winPer','horse_newDistance','horse_newHorse',
#         'horse_last4SpeedRate','horse_latestSpeedRate','Days_from_lastRace','weightRadio','weightRD',
#           'horseFx','horseFx_Distance','jockeyFx','jockeyFx_Distance','propByWinOdds']
        
features=['draw','horse_newDistance','horse_newHorse',
         'horse_last4SpeedRate','horse_latestSpeedRate','Days_from_lastRace','weightRD',
           'horseFx','horseFx_Distance','jockeyFx','jockeyFx_Distance','propByWinOdds']

target_fea='finishTime'
target=[target_fea]
sql = 'select raceDate,raceSeqOfDay,distance,raceClass, horseId, place,lbs,winOdds,'
for aFeature in features:
    sql = sql + aFeature +", "
sql = sql+ target_fea+ " from racecard"
#where racedate='20190213'"

columns=['raceDate','raceSeqOfDay','distance','raceClass',*features,*target]

dataset0 = pd.read_sql(sql,mydb)
#print(dataset0)
#allDates =['20190217']
allDates= dataset0.raceDate.unique()
allDates.sort()
tableName='racecard'


from time import gmtime, strftime
updatedate = strftime("%Y%m%d%H%M%S", gmtime())

In [14]:
count=0
for raceDate in allDates:
    print(raceDate)
    dataset_ofADay = dataset0[(dataset0['raceDate']==raceDate)]
    
    allSeqs = dataset_ofADay.raceSeqOfDay.unique()

    for seq in allSeqs:
        dataset_test =  dataset_ofADay[(dataset_ofADay['raceSeqOfDay']==seq)]
        distance = dataset_test['distance'].iloc[0]
        raceClass = dataset_test['raceClass'].iloc[0]
        #print(raceDate,seq,distance,raceClass)
        count =count+1
        dataset_train = dataset0[(dataset0['raceClass']==raceClass) & (dataset0['distance']==distance) 
                        & (dataset0['raceDate']!=raceDate)]
        #dataset_train.shape
        if(dataset_train.size==0):
            continue
        #regression model of lbw or finishTime
        X_train=dataset_train[features]
        y_train=dataset_train[target] #lbw

        X_test=dataset_test[features]
        y_test=dataset_test[target] #lbw


        reg = LinearRegression()
        reg.fit(X_train,y_train)
        y_test_predicted=reg.predict(X_test)
        
        #plt.scatter(y_test, y_test_predicted,  color='black')
        #plt.show()
        #print("Mean squared error: %.2f"
        #        % mean_squared_error(y_test, y_test_predicted))
        # Explained variance score: 1 is perfect prediction
        #print('Variance score: %.2f' % r2_score(y_test, y_test_predicted))
        #print('explained_variance_score score: %.2f' % explained_variance_score(y_test, y_test_predicted))
         #explained_variance_score(y_test, y_test_predicted)
        df = pd.DataFrame(data=y_test_predicted, index=y_test.index,columns=['finishTime_predicted'])
        dataset_test_toshow=dataset_test[['raceDate','raceSeqOfDay','distance','raceClass','horseId','place','lbs','finishTime']]
        dataset_test_withPredicted=dataset_test_toshow.merge(df,how='inner',left_index=True,right_index=True)
        dataset_test_withPredicted = dataset_test_withPredicted.sort_values(['finishTime_predicted'])
        #print(dataset_test_withPredicted)
       
        predicted_place=1
        for index, row in dataset_test_withPredicted.iterrows():
            mycursor = mydb.cursor()
            raceDate_str = str(row['raceDate'])
            raceSeqOfDate =str(row['raceSeqOfDay'])
            horseId = str(row['horseId'])
            finishTime_predicted =str(row['finishTime_predicted'])
            ##print("Andrea:",raceDate_str,raceSeqOfDate,horseId,finishTime_predicted)
            sql = ("UPDATE "+tableName+ " SET predicted_date=" + updatedate +", predicted_place="+ str(predicted_place) +", predicted_finishTime = "+ finishTime_predicted +
            " WHERE raceDate = "+ raceDate_str + " and raceSeqOfDay = "+raceSeqOfDate + " and horseId ='"+horseId+ "' ")
            
            #print(predicted_place)
            predicted_place =predicted_place+1
            #print(sql)
            mycursor.execute(sql)
            mydb.commit()
            #print(mycursor.rowcount, " record(s) updated" ,row['raceSeqOfDay'] ,",", row['horseId'] )

    print("total record:", count)

20170903
total record: 10
20170906
total record: 18
20170910
total record: 28
20170913
total record: 36
20170916
total record: 46
20170920
total record: 54
20170924
total record: 64
20170927
total record: 72
20171001
total record: 82
20171005
total record: 90
20171008
total record: 100
20171011
total record: 108
20171014
total record: 118
20171018
total record: 126
20171022
total record: 136
20171025
total record: 144
20171029
total record: 154
20171101
total record: 162
20171105
total record: 172
20171108
total record: 180
20171111
total record: 190
20171115
total record: 198
20171119
total record: 208
20171122
total record: 216
20171126
total record: 226
20171129
total record: 234
20171203
total record: 244
20171206
total record: 253
20171210
total record: 263
20171213
total record: 271
20171217
total record: 281
20171220
total record: 289
20171223
total record: 299
20171227
total record: 307
20180101
total record: 318
20180107
total record: 328
20180110
total record: 336
20180113
to